# Estratificando riscos usando registros eletrônicos de pacientes diabéticos

Autor: Rodrigo de Lima Oliveira

Referências:

[DSA] https://www.datascienceacademy.com.br/

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib as m
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

from xgboost import XGBClassifier
import lightgbm as lgb
import sklearn
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.metrics import roc_auc_score
from scikitplot.metrics import plot_confusion_matrix, plot_roc

# Imports para formatação dos gráficos
plt.style.use('fivethirtyeight')
m.rcParams['axes.labelsize'] = 14
m.rcParams['xtick.labelsize'] = 12
m.rcParams['ytick.labelsize'] = 12
m.rcParams['text.color'] = 'k'
from matplotlib.pylab import rcParams 
rcParams['figure.figsize'] = 14,6

In [ ]:
# Carregando os dados
df = pd.read_csv("data/diabetes.csv")

# Exibindo primeiros registros do dataset
df.head()

Dicionário de Dados:
- 0- **encounter_id** - identificador único de um encontro do pesquisador com o paciente.
- 1- **patient_nbr** - identificador exclusivo de um paciente.
- 2- **race** - valores: Caucasian, Asian, African American, Hispanic e other.
- 3- **gender** - valores: male, female, and unknown/invalid.
- 4- **age** - agrupados em intervalos de 10 anos: (0, 10), (10, 20), ..., (90, 100).
- 5- **weight** - peso em libras.
- 6- **admission_type_id** - identificador inteiro correspondente a 9 valores distintos, por exemplo, "emergência, urgência, eletiva, recém-nascido e não disponível".
- 7- **discharge_disposition_id** - identificador inteiro correspondente a 29 valores distintos, por exemplo, "enviado para casa, expirou e não está disponível".
- 8- **admission_source_id** - identificador inteiro correspondente a 21 valores distintos, por exemplo, "encaminhamento médico, e transferência de um hospital".
- 9- **time_in_hospital** - número inteiro de dias entre a admissão e a alta,
- 10- **payer_code** - identificador inteiro correspondente a 23 valores distintos, por exemplo, Blue Cross / Blue Shield, Medicare e auto-pagamento".
- 11- **medical_specialty** - identificador inteiro de uma especialidade do médico admitidor, correspondente a 84 valores distintos, por exemplo, cardiologia, medicina interna, família / clínica geral e cirurgião".
- 12- **num_lab_procedures** - número de testes de laboratório realizados durante a consulta.
- 13- **num_procedures** - número de procedimentos (exceto testes de laboratório) realizados durante a consulta.
- 14- **num_medications** - número de medicamentos genéricos distintos administrados durante a consulta.
- 15- **number_outpatient** - número de consultas ambulatoriais do paciente no ano anterior a consulta.
- 16- **number_emergency** - número de visitas de emergência do paciente no ano anterior a consulta.
- 17- **number_inpatient** - número de visitas hospitalares do paciente no ano anterior a consulta.
- 18- **diag_1** - diagnóstico primário (codificado como três primeiros dígitos da CID9); 848 valores distintos.
- 19- **diag_2** - diagnóstico secundário (codificado como três primeiros dígitos da CID9); 923 valores distintos.
- 20- **diag_3** - diagnóstico secundário adicional (codificado como três primeiros dígitos da CID9); 954 valores distintos.
- 21- **number_diagnoses** - número de diagnósticos inseridos no sistema.
- 22- **max_glu_serum** - teste sérico de glicose que indica a faixa do resultado ou se o teste não foi realizado. Valores: > 200, > 300, normal e nenhum, se não for medido.
- 23- **A1Cresult** - teste A1c que indica o intervalo do resultado ou se o teste não foi realizado. Valores: > 8 (se o resultado for maior que 8%), > 7 (se o resultado for maior que 7%, porém menor que 8%), normal (se o resultado for inferior a 7%) e nenhum, se não for medido.

Na sequência temos os recursos de 24 a 46 para os nomes dos medicamentos genéricos:

metformina, repaglinida, nateglinida, clorpropamida, glimepirida, acetohexamida, glipizida, gliburida, tolbutamida, pioglitazona, rosiglitazona, acarbose, miglitol, troglitazona, insulazforamida, examide, sitaglagliptida, sitazagliptida , glipizida-metformina, glimepirida-pioglitazona, metformina-rosiglitazona e metformina-pioglitazona, 

Cada um desses recursos indica se o medicamento foi prescrito ou se houve uma alteração na dosagem. Valores: "up" se a dose foi aumentada durante a consulta. 

- 47- **change** - indica se houve alteração nos medicamentos para diabéticos (dosagem ou nome genérico). Valores: "change" e "no change".
- 48- **diabetesMed** - indica se houve algum medicamento diabético prescrito. Valores: "sim" e "não".
- 49- **readmitted** - readmitido, "Dias para readmissão hospitalar. Valores: <30 (se o paciente foi readmitido em menos de 30 dias), > 30 (se o paciente foi readmitido em mais de 30 dias) e No, para nenhum registro de readmissão. 

Nota: Códigos ICD-9 ou CID-9 (International Classification of Diseases ou Código Internacional de Doenças).

## Realizando Engenharia de Atributos

In [ ]:
def feature_engineering(dados):

    # Removendo as 3 colunas com alto percentual de valores ausentes
    dados = dados.drop(['weight','payer_code','medical_specialty'], axis = 1)

    # Removemos os registros com baixo percentual de valores ausentes
    dados = dados[dados['race'] != '?']
    dados = dados[dados['diag_1'] != '?']
    dados = dados[dados['diag_2'] != '?']
    dados = dados[dados['diag_3'] != '?']
    dados = dados[dados['gender'] != 'Unknown/Invalid']

    # Removendoo variáveis com valores únicos
    dados = dados.loc[:, dados.nunique() != 1]

    # Ajustando a variável alvo
    # '0' significa que não foi readmitido
    # '1' significa que foi readmitido, não importa quantos dias após a alta
    dados['readmitted'] = dados['readmitted'].replace('>30', 1)
    dados['readmitted'] = dados['readmitted'].replace('<30', 1)
    dados['readmitted'] = dados['readmitted'].replace('NO', 0)

    # Recategorizamos 'idade' para que a população seja distribuída de maneira mais uniforme
    # Classificamos como faixa de 0-50 pacientes de até 50 anos
    dados['age'] = pd.Series(['[0-50)' if val in ['[0-10)', '[10-20)', '[20-30)', '[30-40)', '[40-50)'] else val 
                              for val in dados['age']], index = dados.index)

    # Acima de 80 anos ficam na faixa de 80-100
    dados['age'] = pd.Series(['[80-100)' if val in ['[80-90)', '[90-100)'] else val 
                              for val in dados['age']], index = dados.index)


    # A variável 'admission_type_id' contém 8 níveis
    # Reduziremos os níveis de 'admission_type_id' para duas categorias
    dados['admission_type_id'] = pd.Series(['Emergencia' if val == 1 else 'Outro' 
                                            for val in dados['admission_type_id']], index = dados.index)


    # A variável 'discharge_disposition_id' contém 26 níveis
    # Reduziremos os níveis de 'discharge_disposition_id' para duas categorias
    dados['discharge_disposition_id'] = pd.Series(['Casa' if val == 1 else 'Outro' 
                                                  for val in dados['discharge_disposition_id']], index = dados.index)


    # A variável 'admission_source_id' contém 17 níveis
    # # Reduziremos os níveis de 'admission_source_id' para três categorias
    dados['admission_source_id'] = pd.Series(['Sala_Emergencia' if val == 7 else 'Recomendacao' if val == 1 else 'Outro' 
                                                  for val in dados['admission_source_id']], index = dados.index)


    # Concatena 3 variáveis em um dataframe
    diagnostico = dados[['diag_1', 'diag_2', 'diag_3']]

    # Aplicamos a função comorbidade aos dados
    dados['comorbidade'] = diagnostico.apply(calcula_comorbidade, axis = 1)

    # Drop das variáveis individuais
    dados.drop(['diag_1','diag_2','diag_3'], axis = 1, inplace = True)
    
    # Removendo dataframe temporario
    del diagnostico

    # Lista com os nomes das variáveis de medicamentos (3 variáveis já tinham sido removidas)
    medicamentos = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 
                    'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 
                    'troglitazone', 'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin', 
                    'glimepiride-pioglitazone', 'metformin-pioglitazone']


    # Loop para ajustar o valor das variáveis de medicamentos
    for col in medicamentos:
        if col in dados.columns:
            colname = str(col) + 'temp'
            dados[colname] = dados[col].apply(lambda x: 0 if (x == 'No' or x == 'Steady') else 1)


    # Cria uma variável para receber a contagem por paciente
    dados['num_alt_dosagem_med'] = 0

    # Contagem de modificações na dosagem de medicamentos
    for col in medicamentos:
        if col in dados.columns:
            colname = str(col) + 'temp'
            dados['num_alt_dosagem_med'] = dados['num_alt_dosagem_med'] + dados[colname]
            del dados[colname]


    # Recoding das colunas de medicamentos
    for col in medicamentos:
        if col in dados.columns:
            dados[col] = dados[col].replace('No', 0)
            dados[col] = dados[col].replace('Steady', 1)
            dados[col] = dados[col].replace('Up', 1)
            dados[col] = dados[col].replace('Down', 1) 


    # Variável com a contagem de medicamentos por paciente
    dados['num_med'] = 0

    # Carregamos a nova variável
    for col in medicamentos:
        if col in dados.columns:
            dados['num_med'] = dados['num_med'] + dados[col]


    # Remove as colunas de medicamentos
    dados = dados.drop(columns = medicamentos)


    # Recoding de variáveis categóricas binárias
    dados['change'] = dados['change'].replace('Ch', 1)
    dados['change'] = dados['change'].replace('No', 0)
    dados['gender'] = dados['gender'].replace('Male', 1)
    dados['gender'] = dados['gender'].replace('Female', 0)
    dados['diabetesMed'] = dados['diabetesMed'].replace('Yes', 1)
    dados['diabetesMed'] = dados['diabetesMed'].replace('No', 0)


    # Recoding de variáveis categóricas (label encoding)
    dados['A1Cresult'] = dados['A1Cresult'].replace('>7', 1)
    dados['A1Cresult'] = dados['A1Cresult'].replace('>8', 1)
    dados['A1Cresult'] = dados['A1Cresult'].replace('Norm', 0)
    dados['A1Cresult'] = dados['A1Cresult'].replace('None', -99)
    dados['max_glu_serum'] = dados['max_glu_serum'].replace('>200', 1)
    dados['max_glu_serum'] = dados['max_glu_serum'].replace('>300', 1)
    dados['max_glu_serum'] = dados['max_glu_serum'].replace('Norm', 0)
    dados['max_glu_serum'] = dados['max_glu_serum'].replace('None', -99)


    # Removendo duplicidades por id de paciente, mantendo o primeiro registro
    dados = dados.drop_duplicates(subset = ['patient_nbr'], keep = 'first')


    # Remove as variáveis de ID
    dados.drop(['encounter_id', 'patient_nbr'], axis = 1, inplace = True)
    
    return dados

In [ ]:
# Função que calcula a Comorbidade
def calcula_comorbidade(row):
    
    # Código 250 indica diabetes
    codigos_doenca_diabetes = "^[2][5][0]"
    
    # Códigos 39x (x = valor entre 0 e 9)
    # Códigos 4zx (z = valor entre 0 e 6 e x = valor entre 0 e 9)
    # Esses códigos indicam problemas circulatórios
    codigos_doenca_circulatorios = "^[3][9][0-9]|^[4][0-5][0-9]"
    
    # Inicializa variável de retorno
    valor = 0
    
    # Valor 0 indica que:
    # Diabetes E problemas circulatórios não foram detectados de forma simultânea no paciente
    if(not(bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_1']))))) and
       not(bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_2']))))) and 
       not(bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_3'])))))) and (not(
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_1']))))) and not(
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_2']))))) and not(
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_3'])))))):
        valor = 0
        
    # Valor 1 indica que:
    # Pelo menos um diagnóstico de diabetes E problemas circulatórios foram detectados de forma 
    # simultânea no paciente
    if(bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_1'])))) or 
       bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_2'])))) or 
       bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_3']))))) and (not(
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_1']))))) and not(
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_2']))))) and not(
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_3'])))))): 
        valor = 1
        
    # Valor 2 indica que:
    # Diabetes E pelo menos um diagnóstico de problemas circulatórios foram detectados de forma 
    # simultânea no paciente
    if(not(bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_1']))))) and
       not(bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_2']))))) and 
       not(bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_3'])))))) and (
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_1'])))) or 
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_2'])))) or 
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_3']))))):
        valor = 2
        
    # Valor 3 indica que:
    # Pelo menos um diagnóstico de diabetes e pelo menos um diagnóstico de problemas circulatórios 
    # foram detectados de forma simultânea no paciente
    if(bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_1'])))) or 
       bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_2'])))) or 
       bool(re.match(codigos_doenca_diabetes, str(np.array(row['diag_3']))))) and (
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_1'])))) or 
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_2'])))) or 
        bool(re.match(codigos_doenca_circulatorios, str(np.array(row['diag_3']))))):
        valor = 3 
    
    return valor

## Preparando os dados para treinamento

In [ ]:
# Funcao para balanceamento entre as classes
def balanceamento_classe(X, y):

    # Extraímos os nomes das colunas das variáveis preditoras
    col = df.columns

    # Criamos o objeto SMOTE
    sm = SMOTE(random_state = 42)

    # Treinamos o balanceador SMOTE
    new_X, new_y = sm.fit_sample(X, y)   
    
    return new_X, new_y, col

In [ ]:
# Aplicando engenharia de atributos no dataset
train = feature_engineering(df)

# Vamos armazenar 'readmitted' no rótulo (y) e o restante das colunas em X
y = train['readmitted']
X = train.drop(['readmitted'], axis = 1)

# Vamos criar variáveis dummy para variáveis categóricas
X = pd.get_dummies(X)

# Cria o padronizador
scaler = StandardScaler()

# Padronizando os dados
X = scaler.fit_transform(X)

# Aplicar o balanceamento de classes usando SMOTE
new_X, new_y, features_columns = balanceamento_classe(X, y)

## Executando vários modelos de classificação

In [ ]:
# Definindo os valores para o número de folds
num_folds = 10
seed = 42

# Preparando a lista de modelos
modelos = []
modelos.append(('LR',  LogisticRegression()))
modelos.append(('RF',  RandomForestClassifier()))
modelos.append(('KNN', KNeighborsClassifier()))
modelos.append(('DT',  DecisionTreeClassifier()))
modelos.append(('NB',  GaussianNB()))
modelos.append(('XGB', XGBClassifier()))
modelos.append(('LGB', lgb.LGBMClassifier()))

# Avaliando cada modelo em um loop
resultados = []
nomes      = []

for nome, modelo in modelos:
    kfold = KFold(n_splits = num_folds, shuffle=True, random_state = seed)
    cv_results = cross_val_score(modelo, X, y, cv = kfold, scoring = 'accuracy')
    resultados.append(cv_results)
    nomes.append(nome)
    msg = "%s: %f (%f)" % (nome, cv_results.mean(), cv_results.std())
    print(msg)

In [ ]:
# Boxplot para comparar os algoritmos
fig = plt.figure(figsize = (18, 6))

fig.suptitle('Comparação do desempenho dos Algoritmos de Classificação')
ax = fig.add_subplot(111)
plt.boxplot(resultados)
ax.set_xticklabels(nomes)
plt.show()

## Realizando Otimização de Hyperparametros no melhor modelo

In [ ]:
def plot_prediction_distribution(y_true, y_pred, ax):
    df = pd.DataFrame({'prediction': y_pred, 'original': y_true})    
    sns.distplot(df[df['original'] == 0]['prediction'], label='negative', ax=ax)
    sns.distplot(df[df['original'] == 1]['prediction'], label='positive', ax=ax)
    ax.legend(prop={'size': 16}, title = 'Labels')
    
# Definindo os valores para o número de folds
N_SPLITS  = 3
SEED      = 0

TRAIN_PARAMS = {
                'num_boosting_rounds': 1000000,
                'early_stopping_rounds' : 4000
               }

MODEL_PARAMS = {
                 'boosting_type'           : 'gbdt',
                 'metric'                  : 'auc',
                 'objective'               : 'binary',
                 'learning_rate'           : 0.01,
                 'max_depth'               : -1,
                 'min_data_in_leaf'        : 260,
                 'min_sum_hessian_in_leaf' : 10.0,
                 'num_leaves'              : 10,
                 'verbosity'               : 1
               }

folds = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
oof = np.zeros(len(new_X))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(new_X, new_y)):

    print("Fold {}".format(fold_))

    trn_data = lgb.Dataset(new_X[trn_idx], label=new_y[trn_idx])
    val_data = lgb.Dataset(new_X[val_idx], label=new_y[val_idx])

    clf = lgb.train(MODEL_PARAMS, 
                    trn_data, 
                    TRAIN_PARAMS['num_boosting_rounds'], 
                    valid_sets = [trn_data, val_data], 
                    verbose_eval = 5000, 
                    early_stopping_rounds = TRAIN_PARAMS['early_stopping_rounds'])

    oof[val_idx] = clf.predict(new_X[val_idx], num_iteration=clf.best_iteration)            
    
roc_auc_oof = roc_auc_score(new_y, oof)
print("CV ROC_AUC score: {:<8.5f}".format(roc_auc_oof))

preds = pd.DataFrame(oof, columns=['pos_preds'])
preds['neg_preds'] = 1.0 - preds['pos_preds']

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(24, 6))

plot_prediction_distribution(new_y, preds['pos_preds'], ax=ax1);

plot_roc(new_y, preds[['neg_preds','pos_preds']], ax=ax2);

plot_confusion_matrix(new_y, oof > 0.5, ax=ax3);

## Visualização do Desempenho do Modelo

In [ ]:
roc_auc_oof = roc_auc_score(new_y, oof)
print("CV ROC_AUC score: {:<8.5f}".format(roc_auc_oof))

preds = pd.DataFrame(oof, columns=['pos_preds'])
preds['neg_preds'] = 1.0 - preds['pos_preds']

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(24, 6))

plot_prediction_distribution(new_y, preds['pos_preds'], ax=ax1);

plot_roc(new_y, preds[['neg_preds','pos_preds']], ax=ax2);

plot_confusion_matrix(new_y, oof > 0.5, ax=ax3);

## Estratificação de Risco

In [ ]:
# Função para estratificar o risco
def classifica_risco(row):
    if row[0] <= 0.3 : return 'Baixo'
    if row[0] >= 0.7 : return 'Alto'
    return 'Médio'

# Criando um novo dataset com as previsões
df_proba = pd.DataFrame(oof, columns = ['Probabilidade'])

# Dataframe para o risco estratificado
df_risco = pd.DataFrame()

# Agora carregamos o dataframe
df_risco['Risco'] = df_proba.apply(classifica_risco, axis = 1)

# Vamos separar em percentual
percentual = round(df_risco.Risco.value_counts() / len(df_risco.index) * 100, 1)

# Legenda para o gráfico
sub = ['Médio', 'Alto', 'Baixo']

# Plot
plt.axis("equal")
plt.pie(percentual , 
        labels = sub, 
        radius = 1.6,
        autopct = '%1.2f%%',
        explode = [0.09,0.09,0.09],
        startangle = 30,
        shadow = True,
        counterclock = False,
        pctdistance = 0.8)

plt.show()